In [ ]:
pip install transformers

In [ ]:
pip install torchtext

In [ ]:
pip install -U torchtext==0.8.0

In [ ]:
pip install preprocessor

In [ ]:
pip install ktrain

In [ ]:
pip install tensorflow

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import nltk
import os
import collections
import re
import string

import preprocessor as p
import ktrain
from ktrain import text
import nltk
from sklearn.preprocessing import LabelEncoder


In [2]:
df = pd.read_csv("preprocess_data.csv",index_col=0)
df= df.dropna()
df.head()

,Unnamed: 0.1,_id,text,task_1,task_2,text_clean
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF,PRFN,made amp amp onli abl start make money sustain...
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,OFFN,technic still turn back clock dick head
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT,NONE,govt stop think world media liber gang ani opt...
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,OFFN,soldier japan dick head
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,OFFN,would better ask think sleazi shitbag lmao


In [5]:
LE = LabelEncoder()
df['task_1'] = LE.fit_transform(df['task_1'])
task1_params = dict(zip(LE.classes_, LE.transform(LE.classes_)))
df.head()


,Unnamed: 0.1,_id,text,task_1,task_2,text_clean
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,0,PRFN,made amp amp onli abl start make money sustain...
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,0,OFFN,technic still turn back clock dick head
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,1,NONE,govt stop think world media liber gang ani opt...
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,0,OFFN,soldier japan dick head
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,0,OFFN,would better ask think sleazi shitbag lmao


In [6]:
LE.fit(df['task_1'])
classes= list(LE.classes_)
classes

[0, 1]

## Splitting this dataset into three sets – train, validation, and test.

In [7]:
# split train dataset into train, validation and test sets
train_text, temp_text, train_labels, temp_labels = train_test_split(df['text_clean'], df['task_1'], 
                                                                    random_state=42, 
                                                                    test_size=0.1)

In [8]:
X_train = train_text.tolist()
X_test = temp_text.tolist()

y_train = train_labels.tolist()
y_test = temp_labels.tolist()

In [9]:
y_test[:10]

[0, 1, 1, 0, 0, 0, 1, 0, 1, 0]

## Training the data and validating the model.

In [13]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'  # replace this with model of choice
t = text.Transformer(MODEL_NAME, maxlen=500, class_names=classes)
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)
history = learner.fit_onecycle(5e-5,1)
learner.validate(class_names=classes) # class_names must be string values

preprocessing train...
language: en
train sequence lengths:
	mean : 14
	95percentile : 27
	99percentile : 31


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 14
	95percentile : 27
	99percentile : 31




begin training using onecycle policy with max lr of 5e-05...


Downloading:  16%|█▌        | 73.7k/466k [2:04:13<11:01:00, 9.89B/s]


576/576 [==============================] - 7943s 14s/step - loss: 0.5404 - accuracy: 0.7466 - val_loss: 0.4615 - val_accuracy: 0.7786
              precision    recall  f1-score   support

           0       0.81      0.88      0.84       263
           1       0.68      0.56      0.62       121

    accuracy                           0.78       384
   macro avg       0.75      0.72      0.73       384
weighted avg       0.77      0.78      0.77       384



array([[231,  32],
       [ 53,  68]])

In [14]:
# list all data in history
print(history.history.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy', 'lr', 'iterations', 'momentum'])


In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [16]:
learner.view_top_losses(n=1, preproc=t)

----------
id:346 | loss:3.25 | true:1 | pred:0)



In [ ]:
print(X_test[221],y_test[221])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [19]:
predictor.save('bert_A')

In [3]:
reloaded_predictor = ktrain.load_predictor('bert_A')

In [23]:
reloaded_predictor.predict("neutral"),reloaded_predictor.predict("i dont love u bjp")

(1, 0)

In [59]:
df_ans = pd.read_csv("preprocess_test_data.csv",index_col=0)
df_ans['text_clean'] = df_ans['text_clean'].fillna('neutral')
df_ans.head()

,_id,text,text_clean
0,60c5d6bf5659ea5e55deffcb,Fewer people coming in for vaccinations. So sa...,fewer peopl come vaccin sad nurs covidvaccin vumc
1,60c5d6bf5659ea5e55df028c,@MattHancock This may all be true. But... What...,may true piss big dom
2,60c5d6bf5659ea5e55def377,@Layla_EFC I’ve unfollowed him the wanker,unfollow wanker
3,60c5d6bf5659ea5e55def4c7,You guys are losing it all over the world. The...,guy lose world jung label islamophobia
4,60c5d6bf5659ea5e55df01a6,"And thus death laughs... It is sad merriment, ...",thus death laugh sad merriment still covid ind...


In [45]:
df_ans.iloc[448]

_id           60c5d6bf5659ea5e55df021f
text                    @bunniewails 🐀
text_clean                     neutral
Name: 448, dtype: object

In [ ]:
label = []
i = 0
for tweet in df_ans['text_clean'].tolist():
    ans = reloaded_predictor.predict(tweet)
    i += 1
    print(i,len(tweet))
    if (ans == 0):
        label.append('HOF')
    else:
        label.append('NOT')

In [61]:
df_ans['label'] = pd.Series(label)
df_ans.head(20)

,_id,text,text_clean,label
0,60c5d6bf5659ea5e55deffcb,Fewer people coming in for vaccinations. So sa...,fewer peopl come vaccin sad nurs covidvaccin vumc,NOT
1,60c5d6bf5659ea5e55df028c,@MattHancock This may all be true. But... What...,may true piss big dom,HOF
2,60c5d6bf5659ea5e55def377,@Layla_EFC I’ve unfollowed him the wanker,unfollow wanker,HOF
3,60c5d6bf5659ea5e55def4c7,You guys are losing it all over the world. The...,guy lose world jung label islamophobia,NOT
4,60c5d6bf5659ea5e55df01a6,"And thus death laughs... It is sad merriment, ...",thus death laugh sad merriment still covid ind...,NOT
5,60c5d6bf5659ea5e55def837,Oh shooooot. IPL has been suspended 😱 #Cricke...,oh shooooot ipl suspend cricket indiacovidcrisi,NOT
6,60c5d6bf5659ea5e55def6ec,Fuck y’all lmao https://t.co/n9SX9ThxRv,fuck lmao,HOF
7,60c5d6bf5659ea5e55def29e,@narendramodi you are the one to blame for the...,one blame mess india vision solut pr puppet tu...,NOT
8,60c5d6bf5659ea5e55defa01,dont ask me to cook anything aside from poultr...,dont ask cook anyth asid poultri though,HOF
9,60c5d6bf5659ea5e55def464,"""The world suffers a lot. Not because of the v...",world suffer lot becaus violenc bad peopl beca...,NOT


In [62]:
df_ans = df_ans.drop(['text','text_clean'],axis=1)
df_ans = df_ans.reset_index()
df_ans = df_ans.drop(['index'],axis=1)
df_ans.head()

,_id,label
0,60c5d6bf5659ea5e55deffcb,NOT
1,60c5d6bf5659ea5e55df028c,HOF
2,60c5d6bf5659ea5e55def377,HOF
3,60c5d6bf5659ea5e55def4c7,NOT
4,60c5d6bf5659ea5e55df01a6,NOT


In [60]:
df_ans.head()

,_id,text,text_clean
0,60c5d6bf5659ea5e55deffcb,Fewer people coming in for vaccinations. So sa...,fewer peopl come vaccin sad nurs covidvaccin vumc
1,60c5d6bf5659ea5e55df028c,@MattHancock This may all be true. But... What...,may true piss big dom
2,60c5d6bf5659ea5e55def377,@Layla_EFC I’ve unfollowed him the wanker,unfollow wanker
3,60c5d6bf5659ea5e55def4c7,You guys are losing it all over the world. The...,guy lose world jung label islamophobia
4,60c5d6bf5659ea5e55df01a6,"And thus death laughs... It is sad merriment, ...",thus death laugh sad merriment still covid ind...


In [63]:
df_ans.to_csv('BERT_A.csv',index=False)